In [108]:
import ipywidgets as widgets
from IPython.display import display
from stmpy import Machine, Driver
import random

class StudyHall:
    
    def load_images(self):
        self.red_on = open("images/red_on.png", "rb").read()
        self.red_off = open("images/red_off.png", "rb").read()
        self.green_on = open("images/green_on.png", "rb").read()
        self.green_off = open("images/green_off.png", "rb").read()
        
    def on_button_new_card(self, b):
        self.stm.send_signal('newCard')
    
    def display(self):
        self.button_new_card = widgets.Button(description="Swipe new card")
        self.button_new_card.on_click(self.on_button_new_card)
        
        self.red = widgets.Image(value=self.red_off, format='png', width=50, height=50)
        self.green = widgets.Image(value=self.green_off, format='png', width=50, height=50)
        
        self.info = widgets.Label(value="Waiting...")
        self.room_state = widgets.IntProgress(
            value=len(self.id_array),
            min=0,
            max=10,
            step=1,
            description='People:',
            bar_style='info',
            orientation='horizontal'
        )
        
        #self.list_of_ids = [str(i) for i in self.id_array]
        #widgets.HBox(self.list_of_ids)
        
        display(self.red, self.green, self.button_new_card, self.room_state, self.info) #, self.list_of_ids)
        
    def __init__(self):
        self.id_array = []
        
        self.load_images()
        self.display()
        
    #IMAGES:
        
    def turn_red_on(self):
        self.red.set_trait(name='value', value=self.red_on)
        
    def turn_red_off(self):
        self.red.set_trait(name='value', value=self.red_off)
        
    def turn_green_on(self):
        self.green.set_trait(name='value', value=self.green_on)
        
    def turn_green_off(self):
        self.green.set_trait(name='value', value=self.green_off)

    #TIMERS:
        
    def start_idle_timer(self):
        self.stm.start_timer('idle', 5000)
    def destroy_idle_timer(self):
        self.stm.stop_timer('idle')
        
    def start_sample_server_response_timer(self):
        self.stm.start_timer('server', 1000)
    def destroy_sample_server_response_timer(self):
        self.stm.stop_timer('server')
        
    def start_sample_controller_response_timer(self):
        self.stm.start_timer('controller', 1000)
    def destroy_sample_controller_response_timer(self):
        self.stm.stop_timer('controller')
        
    def start_green_display_timer(self):
        self.stm.start_timer('green_display', 1000)
    def destroy_green_display_timer(self):
        self.stm.stop_timer('green_display')
        
    def start_red_display_timer(self):
        self.stm.start_timer('red_display', 1000)
    def destroy_red_display_timer(self):
        self.stm.stop_timer('red_display')
        
    #INFORMATION:
        
    def info_card_check(self):
        self.info.set_trait(name='value', value="Validating ID.")
        
    def info_card_add(self):
        self.info.set_trait(name='value', value="ID added to the system.")
        
    def info_card_remove(self):
        self.info.set_trait(name='value', value="ID removed from the system.")
    
    def info_server_transition(self):
        self.info.set_trait(name='value', value="Updating Server.")
        
    def info_waiting_transition(self):
        self.info.set_trait(name='value', value="Waiting...")
        
    def info_green_state(self):
        self.info.set_trait(name='value', value="Thank you.")
        
    def info_red_state(self):
        self.info.set_trait(name='value', value="There was a mistake, please try again.")
        
    #def info_update_label_array(self):
        #self.list_of_ids = [str(i) for i in self.id_array]
        #widgets.HBox(self.list_of_ids)
        
    def info_update_bar(self):
        self.room_state.set_trait(name='value', value=len(self.id_array))
        
    #STATE FUNCTIONALITY
    
    def state_waiting(self):
        self.turn_red_off()
        self.turn_green_off()
        self.start_idle_timer()
        self.info_waiting_transition()
        
    def state_get_card(self):
        self.turn_red_off()
        self.turn_green_off()
        self.start_sample_controller_response_timer()
        self.info_card_check()
        
    def state_green(self):
        self.turn_red_off()
        self.turn_green_on()
        self.info_green_state()
        self.start_green_display_timer()
        
    def state_red(self):
        self.turn_red_on()
        self.turn_green_off()
        self.info_red_state()
        self.start_red_display_timer()
    
    def state_update_server(self):
        self.turn_red_on()
        self.turn_green_off()
        self.start_sample_server_response_timer()
        self.info_server_transition()
        # here we will send the updated info to the server
        
        
    def function_new_card(self, id): # this represents what we do if we detect a new card
        self.info_card_add()
        print("ID added: ", id, self.id_array)
        self.info_update_bar()    #self.info_update_label_array(id)
    
    def function_old_card(self, id): # this represents what we do if we detect a card already in our system
        self.info_card_remove()
        print("ID removed: ", id, self.id_array)
        self.info_update_bar()    #self.info_update_label_array()
        
        
    # The following is just a random simulation of the predicted behavior
    def get_random_card(self):
        card_id = random.randint(1,10)
        if card_id==1: # we say a card fails in 10% of the cases
            return 'red'
        else:
            if self.id_array.count(card_id)==0:
                self.id_array.append(card_id)
                self.function_new_card(card_id)
                return 'green'
            else :
                self.id_array.remove(card_id)
                self.function_old_card(card_id)
                return 'green'
    

In [109]:
studyHall = StudyHall()

#TRANSITIONS

initiate = {
    'source':'initial',
    'target':'waiting'
}

new_card = {
    'trigger':'newCard',
    'source':'waiting',
    'target':'get_card',
    'effect':'destroy_idle_timer'
    # here we will send the card ID to the controller, and wait for an answer ('effect':'send_ID_to_controller')
}

random_card = { # this is just a simulation of the real behavior
    'trigger':'controller',
    'source':'get_card',
    'function':studyHall.get_random_card
}

green_done = {
    'trigger':'green_display',
    'source':'green',
    'target':'waiting'
}
red_done = {
    'trigger':'red_display',
    'source':'red',
    'target':'waiting'
}



sensor_idle = {
    'trigger':'idle',
    'source':'waiting',
    'target':'update_server'
}
server_done = {
    'trigger':'server',
    'source':'update_server',
    'target':'waiting'
}


#STATES

waiting = {
    'name': 'waiting',
    'entry': 'state_waiting'
}

get_card = {
    'name': 'get_card',
    'entry': 'state_get_card'
    # here we will stay until we receive an answer from the controller, then we change states accordingly
}

green = {
    'name': 'green',
    'entry': 'state_green'
}

red = {
    'name': 'red',
    'entry': 'state_red'
}

update_server = {
    'name': 'update_server',
    'entry': 'state_update_server'
}


stm_studyHall = Machine(name='studyHall', transitions=[initiate, sensor_idle, server_done, new_card, random_card, green_done, red_done], obj=studyHall, states=[waiting, update_server, get_card, green, red])
studyHall.stm = stm_studyHall

driver = Driver()
driver.add_machine(stm_studyHall)
driver.start()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00P\x00\x00\x00O\x08\x02\x00\x00\x00\xf3\xf3\xd5\xb4\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00\tpHYs\x00\x00\x04\x9d\x00\x00\x04\x9d\x01|4k\xa1\x00\x00\x02\x0biTXtXML:com.adobe.xmp\x00\x00\x00\x00\x00<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="XMP Core 5.4.0">\n   <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n      <rdf:Description rdf:about=""\n            xmlns:tiff="http://ns.adobe.com/tiff/1.0/">\n         <tiff:ResolutionUnit>2</tiff:ResolutionUnit>\n         <tiff:Compression>5</tiff:Compression>\n         <tiff:Orientation>1</tiff:Orientation>\n         <tiff:PhotometricInterpretation>2</tiff:PhotometricInterpretation>\n      </rdf:Description>\n   </rdf:RDF>\n</x:xmpmeta>\n\x86\xd2\xae$\x00\x00\x04`IDATx\x01\xed\x9a{S\xdaP\x10\xc5I \x89\x08\x11\x14d\x10\x1d\xedk:\xd3\xbf\xfd\xfe\x9f\xa33\x9d\xbet\xb4P\x15\x0b\x86\x10I \xe9\xa1T\x06\xa1\x86\xec\xde{\x85b\x18\x07\xf3\xd8\xdds~w\x13\x12\xc2\xd5NOO3/\xe9\xa5\xbf$\xd81k\n\xbc\xe9\x1dO;\x9cvx\xc3F =\xa47\xac\xa1\x0b8i\x87\x17\x86d\xc36\xa4\x1d\xde\xb0\x86.\xe0\xbc\xb8\x0e\xe7\x16\x86@\xed\x06}4\xca\x85\xa1\x1e\x8e \x13\xea\xd9\xa1\xae\x87\xd9\xacZ\xc9\xc7\xd5\x95\x03\x83p\xa7\xef\xda}\xb7\xe0y[\xfe\xc0\x18\x8dQg_A6{oZn>\xefl\x17\xee\xb6\x0b\xaa\xf9\x95\x01GQ\xb9\xe7T\xba\x1d\xbc\xebQ4K8\xb7\x8c!0\xbc\xbe\xed\xf5\xeb\xb7\xedP\xd3:E\xbb]*\xe3=\xa3is\x91RV\x15\x00GQ\xb5\xdb\xa9\xb7\xaf\xf3\xbeO\xb5\x88\xa1\xd9s\xee\xf0\xe7\x99f\xab\xb2\x7fS*K\xc7\xce6\x1a\r\xaa\xad\x98\xf8b\xdf}wq^\xeb\xfcZ<tc\xb2\x16w!}\xb7\xe7\x94\x1d\xc7\xb3,\xdf0\x17\x03\xd8[\xe4\x01G\xd1\xe1\xf5\xcf\xd7\xcd\x1f\xe6h\xc8v3\x97\x88R8X\xb4(\xc4\xe9-\xab\xd5r\x0e\xe9\xdcp\x88\xc6\xe2<\x9cs,\xbe\x8a\xf3\xb8\xd1\xbe\xb1\xfb\xfd\xcfG\xc7\xc3\x9c\x04\xb7\x12\xae\xc3\xa6\xef\x7f8\xfb\xaa\x82v:^(\x0e\t\x08M\xb7\xb0\x17D\x81\xcd \x80\x95-\x19V\xe2\x19 1f\x0e\x82\xf8\xb0\xa5{\x85\x80\xb3\xa3\xe1\xfb\xf3\xef\xe6P\xdaI\x1bo\x17B\x90\x83h|X\xfc^\x01\xe0(z{y\x91\xf7\x07\xf1\x02r\xf7B\x0e\xa2\x99\xd8\x0b{\xbc"\x1f\xb8~{Sr{\xf1\xd5U\xec\x85(\xa4\xd9\x95\x99\xc0\x96?8\xbc\xbab\xab\n&B\x1a\x06xE\x98\xc0\'\xad\xa6\x9e\x89\xbba\xe4\xb9I\x98\x05i\x18H\x18<\x17\xc6\x01\xb6\xdd\xdeJ\x0e\xe6Y\xeb0`\xbb\xee\xec\x96\x84\xcb\x1c\xe0\x836\xff\x14Jh+I\xd8A\xfb:I\xd8\\\x0c\x19\xd8\x1a\x0cV\xde\xde\t\x03l\xc0\xcc\x1c\xcf\xd2U2p\xe5\xae\xb3\xb4\xe8\xb3\x050\xcc\x90\x81\xf7\xee\xba\xcf\xc6\xb3T\x88a\x86\x06l\x04\x01\xe3[\xeeR\xdf\xec\x00\x98\x81%R:\r\xd8\xf68\x1f\x8c$C\xd4`\xaa%\x1ap\xfe\x9e\xfc!A\x05\xa0\xc6S-\xd1\x80\xf1\x14\x8ejHu<\xd5\x12\r\xd8\x1c\xd2N\x18\xd5\xb4\xa8O\xb5D\x03\xd6\xc3\xf0\x19\x18H\x12TKT\xe0\x95\xdd??5\nzH\xb3D\x03\x0eu%\xcf\x8a\x9f\x82I\xb2\x9dj\x89\nL\x8bO\xe2X0f\xa4\xd3~\xa9\xa1\x01\xf89C\xd0\x9f\xf4\xf4\x80\xf8(\x93\x06\x8c\x1f\x81\xa4;\x16,H\xb5D\x03\xf6\xb6\xd6\x0e\x98j\x89\x06\xec\xe4\x0b\x82\r\x91\x9eN\xb5D\x03\x0e\x0c\x03?sI7\xcd.\x083\xb0DJ\xa7\x01\xa3\xf4\xedN\x89$\xa04\x98a\x86\x0c\xdc\xde)+e \x15g\x98!\x03\x0f,\xab[(\x92l)\n\x86\r\x98\xa1\x16\'\x03C\xa0Y\xd9\xa7\xca\xa8\x88\xe7\xd9\xe0\x00;\x85\xc2\xca\x9b\x0c\x03\xb0\xc1\x18G\x0e0d\xce\xea\x07afe\xf7\xd5\x90\x86\x01\x06-R\x98\xc0\x03\xd3\xba\xdc\xaf\xf1$\xc5\xb3.k5\x18\xe0\xd5a\x02C\xacU\xa9\xae\xe4\xc0\x86hk\xaf\xca\xa3E\x16\x1f\x18\x93.\xbe\x1c\x1ey\xdc\x91\xe69\x86\x1cDE\xe6{\x08\x00g2\xa3l\xee\xd3\xf1+\x9f\xf8}\x85\x87\x8a,\x08A\x0e\xa2\xec\nH\x14\x02\x1e\x9b0\x8c\x8f\'o\xee\xd5\xdfoB\x02B\x90\x13\xa1\x95\x00<f\xfec\xc5\xc9o\x0bZ\x89IG\xf11\xad\x8ca\x953O+\xd4u\xcc\x9a\xd3\xa2\xa8\xe8\xf5\xe5^\xac\xf0\xc0\xaaY\xa9~k\x1c\xc9\x9a\x83)t><\xea\x89\xa6]\xd6\xea\xdd\xa2}\xdcj\x16\x06\xf7\x8fvqW\\k\xeb\xbc~\xd0\xc3\xac4y/9\x1d\x9e\xfa\xc14\xc1\xeb\xf2.\xde\xff9qv\x1a\xb6t\x01\xdf\xfb.j\xf5\xb3zC\xcaa<+\'\xaf\xc3\xd3\xaa\x9avS\xde\xc5\x11\x9ep6\xed4\x0f\x0b\xff\xe7l\xda\t\x01&\x02\xdb;\xf8{1\xf3\xa5\x1f\x1a\x87\x0f\x9b\t\xf9d\xc3\xe6\xcf\x88\x7f\x00\xff\xfb\x1f\xfc\xfex\xce\xbf\xe8\xe5t\xael\xf2U\xd1\x1b\x8f\xe4Jk\x12\x99\x02\xafI#\x94\xd9H;\xaclh\xd7\xa4p\xda\xe15i\x8

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00P\x00\x00\x00O\x08\x02\x00\x00\x00\xf3\xf3\xd5\xb4\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00\tpHYs\x00\x00\x04\x9d\x00\x00\x04\x9d\x01|4k\xa1\x00\x00\x02\x0biTXtXML:com.adobe.xmp\x00\x00\x00\x00\x00<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="XMP Core 5.4.0">\n   <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n      <rdf:Description rdf:about=""\n            xmlns:tiff="http://ns.adobe.com/tiff/1.0/">\n         <tiff:ResolutionUnit>2</tiff:ResolutionUnit>\n         <tiff:Compression>5</tiff:Compression>\n         <tiff:Orientation>1</tiff:Orientation>\n         <tiff:PhotometricInterpretation>2</tiff:PhotometricInterpretation>\n      </rdf:Description>\n   </rdf:RDF>\n</x:xmpmeta>\n\x86\xd2\xae$\x00\x00\x04\x08IDATx\x01\xed\x9a}O\x1aA\x10\xc6woE\xb0\xa8\x88\xb6\xd6bjb\xabi\xfb\xb7\xdf\xff{\xb4\xa9\xad\xa9\xc6\xc6b}\x85C\x11\xee\xb6\x8f\xbd\x84\xe0a\xc8\xcd\xec,\x9cp\x86\x90\xbbew\x9e\xe77so\xc8\xe8\x83\x83\x035O\x7f\xc1<\xc1>\xb2\x16\xc0\xb3^\xf1\xa2\xc2E\x85g,\x03\xc5!=c\x05\x1d\xc1)*<\x92\x92\x19\x1b(*<c\x05\x1d\xc1\x99\xbb\n/\x8c\xa4\xc0\xef@d\xa3H\xe1\x15C\xc6\xa8\xc0(c\xb4\xf1+\xf94\xbaw`\x10\xb6l\xbbe\xc3\xd0\x86]\xd5\xed\xab\xe8\xa9\x01\xb5\xa0LY\x95\xab\xba\xba\xf2\xf8Z\xf6\xcd\xef\x0b\xd8Z{co/\xec\x15\xde\xad\xb2)\xc8\xe1]\xa4\xa0\xaf:\xa1\xed4\xed\xb9V\xba\xa6W7t\x1d\xefZ\xeb\xe1iR\xdb\xf2\xc0@\x05\xe7Y\xdcD=\xa9.\x91\x9ak{\x83\x17j\xbe\x15l\x82\\\x1c[\x18\xb8m\xc3\xe3\xe8\xf4N\xddQQS\xf3\x91\xac_\xf1IS\xfd\xdd1\xdb\xcb\xba\x9a\xfa\xd4e\xd74\x1a\r\x97\xf5\x83\xb5(\xec\xef\xf8\x0c.\xfb\xaa?\x18t\xdc@\xa8\x0b{\x89\xc88\xb7\xa5J-S\xe1\xbe\xed\x1fFG\xa1\xea8\x12>\xbb\xfc\xcc6[Q{\xcf\xec.h\x01\xb7\x02\xf7\xe1\xae\xed~\x8d\x0e=\xd1&)@pH@\xe8\xd9\x8c\x90\x06]\x81\x1f\xec\xc3\xb7\xe8\x07\xe3\xfaDr\x89\xc9\x90\x80\x10\xe4\xa8\x0bS\xf3\x9d\x80q$\x7f\x8f~\xf6T/\x15\xd4\xd3.\x84 \x07Q\x97\xf8|`\\K\x8e\xe2\xe3{\xfa\xbd\xc7\xc5.\xe4 \niv\x10>\xf0\x1f{~k[la\xf6B\x88B\x9a\xbd\x9c\t|o\xbb\xb8\t\xb1U\x1d\x17B\x1a\x06xA\x98\xc0\'\xf1\xe9\xf8\x07F\x9e\x9b\x8c\xab \r\x03\x19\'\xa7\xa6q\x80\xf1e`*\x07\xf3\xb0u\x18\x80\x8d\xe1\x91\x8c\xdb\x1c`<\'g\x8c\xeeu\x1a\xcf\x06\x19\x18\'\xcf\xd4\xcb\x9b\xe4\x116\x18g2\x19\xf82\xbe\xf2Z7Rp\x86\x192\xf0\x95\xbd&y\xf2:\x99a\x86\x06\xdc\xb3\xbd\t?i\x8c\xcf\x17\xcc\xc0\xd2\xf89\xa9Oi\xc0\xbc\x0bcJRv\x97j\x89\x06|g\xefe\xed\xbaG\xa3Z\xa2\x01O\xe0[\x115\x05TK4\xe0\x07\xe2\tCu\xcf\x98O\xb5D\x03\x8e\xff\xff?\x99a\xcb\xdf\x12\xaa\xa5\x02xl-\x82\xfcu\xfdP-\xd1*L\x8d>6{2\x1fR-\xd1\x80\x17uI\xc6\xa6\\\x14\xaa%\x1a0~\x10\x90\xb3*\x13\x89j\x89\x06\xbc\xa4+26\xe5\xa2P-\xd1\x80\xf1\x0b\x80\x9cU\x99HTK4\xe0\x92.U\xf2tT\xc3\x0c,\x912G\x03F\xe8\xba^#\tx\x9d\xcc0C\x06^\x0f\xea^\x19H\xc1\x19f\xc8\xc0\x15]^\xd5+$[\x9e&\xc3\x06\xccP\x83\x93\x81!\x80\x9f\xaa\xa92>\xe6\xf3lp\x80qa\x9cz\x91a\x80z}N\x92\xce\x01\xc6\xca\xf7\xc16\xfa1|\xd4-KLH\xc3@\x96\x99\xa3s\x98\xc08y\x1a\xc1\xd6h\xb8\xc9\x8c@\x9aq\xf6&\xde\x98\xc0X\xfcV\xbf\x99\xca\x81\rQH\xb33\xcb\x07F\xd3\xc5n\xb03\xe1\xe7\x10\xc8A\xd4\xa5\xdf\x83\x0f\x8c\x1c\xa3\xe9b\xdf|()\xda\xb3\x0e\xbb8\x10\x82\x9cc\xa7\x87\x130\xac/\xea\xc5O\xe6#\xf5+\x0b\x83\x19\x12\x10\x82\x1cc\xed\xf0\x12W`\xc4*\xeb\xf2g\xb3WU\xaf\x86\xe3\xcan#8$ \xe4\x1eV\xa6O+\xd0\xc1\x86^\xc7\xcf\xb6m\x15\xba{JE\xd8\xd2\x9b\xbbfG\xaa\x07S\xa0\xf3)\xf1\x87\x0b\xc9\xb6yW\xb3\xab"\x9dxI\xcc%\xb5$\xde\x89\'\x06\x9cXD\x9b\xe0\x17\xb3\xcf\xee\xb5\x1c\xd4\x16g\xec\xcb\xe8\xb5\x84c\x94\xfa\xb5^G_h\xc6n\xda\x01\xe4\xe3\xda\x97\xd8M\x9b\x00\x00{M\xd7\xd6Tm^\xfa\xa5\x07u\xc3\xc5&!OFf\xbf#~@\x9el\x80\x1fm\xff\xa9\xc1I\xee\n\xdc\x87\'i\xd7]\xab\x00v\xcfa\xbe#\x14\x15\xcew}\xdc\xdd\x15\x15v\xcfa\xbe#\x14\x15\xcew}\xdc\xdd\x15\x15v\xcfa\xbe#\x14\x15\xcew}\xdc\xdd\xfd\x03#\xb8\\5=\xdcf@\x00\x00\x00\x00IEND\xaeB`\x82', height='50', width='50')

Button(description='Swipe new card', style=ButtonStyle())

IntProgress(value=0, bar_style='info', description='People:', max=10)

Label(value='Waiting...')

ID added:  7 [7]
ID added:  5 [7, 5]
ID added:  3 [7, 5, 3]
